In [1]:
import pandas as pd
import numpy as np
import os
import random

import warnings

warnings.filterwarnings(action='ignore')

k = 100 # k개까지만 문제 기록을 남기려고함.

path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')
dat = pd.concat([train, test])
dat = dat.loc[::-1]
dat.reset_index(drop= True, inplace= True)

dat['tem'] = 1
dat['seq'] = dat.groupby('userID')['tem'].apply(lambda x : x.cumsum())

In [2]:
# test 데이터 만들기.
test_id = test['userID'].unique()
test = dat[(dat['userID'].isin(test_id)) & (dat['seq'] <= k)]
train = dat[~((dat['userID'].isin(test_id)) & (dat['seq'] == 1))]
train['seq'] = train.apply(lambda x : x['seq'] - 1  if x['userID'] in test_id else x['seq'], axis = 1)

# valid 데이터 만들기.
valid_data = train[train['seq'] <= k]

# 뒷처리부분.
user_num = dat['userID'].nunique()
train_new = train.copy()

# seq 하나씩 내리고 0된 값 처리하기.
train_new['seq'] = train_new['seq'] - 1
train_new = train_new[train_new['seq'] >= 1]

# train_new 데이터에 문제 너무 적게 푼 유저 정리하기.
tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
lst = list(tem[tem['seq'] < 14]['userID'])
train_new = train_new[~train_new['userID'].isin(lst)]

# 첫번째 train_datas 만들기.
train_datas = train_new[train_new['seq'] <= k]

In [3]:
for i in range(30):
    # seq 하나씩 내리고 0된 값 처리하기.
    train_new['seq'] = train_new['seq'] - 25
    train_new = train_new[train_new['seq'] >= 1]
    
    # 문제를 적게 푼 유저 정리하기.
    tem = train_new.drop_duplicates(['userID'], keep='last')[['userID','seq']]
    lst = list(tem[tem['seq'] < 14]['userID'])
    train_new = train_new[~train_new['userID'].isin(lst)]
    train_new['userID'] = train_new['userID'] + user_num
    
    # train_data 만들기.
    train_data = train_new[train_new['seq'] <= k]
    train_datas = pd.concat([train_datas, train_data])

    if i % 5 == 0:
        print(i)

0
5
10
15
20
25


In [4]:
train_data = train_datas.sort_values(['userID','seq'], ascending=[True, False])
valid_data = valid_data.sort_values(['userID','seq'], ascending=[True, False])
test_data = test.sort_values(['userID','seq'], ascending=[True, False])

train_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/train_data3.csv', index = False)
valid_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/valid_data3.csv', index = False)
test_data.drop(['tem'], axis = 1).to_csv(path + 'GKT/test_data3.csv', index = False)

In [7]:
valid_data['userID'].value_counts()

0       100
3463    100
3482    100
3481    100
3480    100
       ... 
7398     14
7252     13
7414     13
7390     13
7441      9
Name: userID, Length: 7442, dtype: int64

In [8]:
train_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,tem,seq
2526055,0,A060127007,A060000127,1,2020-10-20 05:20:05,713,1,100
2526054,0,A060129001,A060000129,1,2020-10-21 05:41:03,714,1,99
2526053,0,A060129002,A060000129,1,2020-10-21 05:41:13,714,1,98
2526052,0,A060129003,A060000129,1,2020-10-21 05:42:16,714,1,97
2526051,0,A060129004,A060000129,1,2020-10-21 05:43:34,714,1,96
...,...,...,...,...,...,...,...,...
554123,227305,A060057005,A060000057,0,2020-05-12 22:42:53,610,1,5
554122,227305,A060057006,A060000057,0,2020-05-12 22:43:08,610,1,4
554121,227305,A060057007,A060000057,0,2020-05-12 22:43:29,610,1,3
554120,227305,A060061001,A060000061,1,2020-05-18 11:10:46,612,1,2


In [9]:
valid_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,tem,seq
2526054,0,A060129001,A060000129,1,2020-10-21 05:41:03,714,1,100
2526053,0,A060129002,A060000129,1,2020-10-21 05:41:13,714,1,99
2526052,0,A060129003,A060000129,1,2020-10-21 05:42:16,714,1,98
2526051,0,A060129004,A060000129,1,2020-10-21 05:43:34,714,1,97
2526050,0,A060129005,A060000129,0,2020-10-21 05:43:42,714,1,96
...,...,...,...,...,...,...,...,...
260118,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,5
260117,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4
260116,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,3
260115,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,2
